# Nutrition

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

## Raw Data

In [22]:
DATA_PATH_AVG = './data/input/nutrients_avg.xlsx'

def process_data(data_path):
    
    df = pd.read_excel(DATA_PATH_AVG)
    
    # 
    df_no_time = df.drop(columns='Time')
    X = df_no_time.to_numpy()

    # Sort data along axis=0 and reverse to get descending order
    data = np.sort(X, axis=0)[::-1]

    # normalize data to 1
    maxes = np.max(data, axis=0)

    data = data / maxes
    return data, maxes

## Data Preparation

In [23]:
# from sklearn.preprocessing import MinMaxScaler

# Convert DataFrame to NumPy array

# If a scaler is applied it also has to be applied to the fertilizer

# scaler = MinMaxScaler()

# # Fit the scaler and transform the data to scale each column independently
# data = scaler.fit_transform(data)

# # Now each column of data is scaled to the range [0, 1]
# print(data)

In [24]:
from scipy.interpolate import interp1d

def interpolate_data(data, num_points=500, kind='cubic'):
    x_original = np.arange(data.shape[0])  # Original index positions
    x_interp = np.linspace(0, data.shape[0] - 1, num_points)  # Interpolation index positions
    # Initialize an array to store interpolated data
    Y_interp = np.zeros((num_points, data.shape[1]))
    # Interpolate each column
    for col in range(data.shape[1]):
        f = interp1d(x_original, data[:, col], kind='cubic')  # Use cubic interpolation for smoothness
        Y_interp[:, col] = f(x_interp)

    return x_interp, Y_interp

x_interp, Y_interp = interpolate_data(data, num_points=4, kind='linear')

NameError: name 'data' is not defined

In [25]:
def plot_curves(x, y):
    # Plot the derivative of the first column
    fig, axes = plt.subplots(4,3, figsize=(12,10))
    axes = axes.flatten()
    for i in range(11):
        axes[i].plot(x,y[:, i])
        axes[i].set_ylim([0,None])

plot_curves(x=x_interp, y=Y_interp)

NameError: name 'x_interp' is not defined

In [26]:
from scipy.interpolate import CubicSpline

def calculate_nutrient_uptake_rate(x_interp, Y_interp, num_interp_points=500):

    # Initialize an array to store the derivatives
    Y_derivative_spline = np.zeros((num_interp_points, data.shape[1]))

    # Compute the cubic spline and its derivative for each column
    for col in range(data.shape[1]):
        cs = CubicSpline(x_interp, Y_interp[:, col])  # Cubic Spline interpolation
        Y_derivative_spline[:, col] = cs.derivative()(x_interp)  # Evaluate the derivative

    plant_uptake_rate = - Y_derivative_spline 
    return x_interp, plant_uptake_rate

x_interp, plant_uptake_rate = calculate_nutrient_uptake_rate(x_interp, Y_interp, num_interp_points=4)

plot_curves(x=x_interp, y=plant_uptake_rate)

NameError: name 'x_interp' is not defined

In [27]:
# def absolute_plant_uptake_from_rate(x_interp, plant_uptake_rate, start, end):
#     start_index = np.argmin(np.abs(x_interp - start)) # find the best index
#     end_index = np.argmin(np.abs(x_interp - end))
#     return np.sum(plant_uptake_rate[start_index:end_index,:],axis=0)/(end_index-start_index)

# absolute_uptake = absolute_plant_uptake_from_rate(x_interp, plant_uptake_rate, 0,1)
# print(f"{absolute_uptake = }")

In [28]:
def absolute_plant_uptake_during_interval(x_interp, Y_interp, start, end):
    start_index = np.argmin(np.abs(x_interp - start)) # find the best index
    end_index = np.argmin(np.abs(x_interp - end))
    # uptake is the negative of nutrient change in solution
    return -(Y_interp[end_index,:] - Y_interp[start_index,:]) 

target = absolute_plant_uptake_during_interval(x_interp, Y_interp, 2,3)
print(f"{target = }")

NameError: name 'x_interp' is not defined

## Optimize

Consider a single time intervall $I = [t_0, T]$.

Unsing the model of the plant uptake we have obtained the the target nutrient concentration should be idealy $\vec T$.

Let us optimize the minimal total amount of fertilizer to reach the target under the constraint that the plant does not experience any deficite of nutrients.

$L = \min(|\vec{T} - (u_1 \vec{F}_1 + u_2 \vec{F}_2 + u_3 \vec{F}_3|)$

Constrain: $\vec{T} - (u_1 \vec{F}_1 + u_2 \vec{F}_2 + u_3 \vec{F}_3) \ge 0$

**Comments:**

1. Since the concentration have different scales, it might be reasonable to scale (transfrom) them using sklearn MaxMinScaler or StandartScaler.
Afterwards don't forget to apply the inverse transform to get correct values!

In [29]:
def nutrients(params, c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target):
    a, b, c = params
    value = c_target - a*c_fertilizer1 - b*c_fertilizer2 - c*c_fertilizer3
    return value  

def objective_function(params, c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target):
    a, b, c = params
    value = np.sum(np.abs(c_target - a*c_fertilizer1 - b*c_fertilizer2 - c*c_fertilizer3))
    return value  

def constraint_function(params, c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target):
    a, b, c = params
    return np.min(c_target - a*c_fertilizer1 - b*c_fertilizer2 - c*c_fertilizer3)

def optimize(c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target, constraint):
    # Initial guess for a and b
    initial_guess = [0, 0, 0]
    bounds = [(0, None), (0, None), (0, None)]  # (lower bound, upper bound) for a, b and c
    
    # Call the optimizer
    result = minimize(objective_function, initial_guess, 
                      args=(c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target),
                      bounds = bounds,  # (lower bound, upper bound) for a and b
                      constraints=[constraint], 
                      method='SLSQP') 
    
    # Extract the optimized values of a and b
    a_opt, b_opt, c_opt = result.x
    return a_opt, b_opt, c_opt, result.fun  # Return optimized a, b, and the minimized value


In [30]:
c_target = np.abs(np.random.sample(11))*10/maxes

c_fertilizer1 = np.abs(np.random.sample(11))/maxes
c_fertilizer2 = np.abs(np.random.sample(11))/maxes
c_fertilizer3 = np.abs(np.random.sample(11))/maxes

# Define the constraint
constraint = {
    'type': 'ineq',  # 'ineq' means the constraint function must return >= 0
    'fun': constraint_function,
    'args': (c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target)
}


a_opt, b_opt, c_opt, minimized_value = optimize(c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target, constraint)
params_opt = (a_opt, b_opt, c_opt)
nut = nutrients(params_opt, c_fertilizer1, c_fertilizer2, c_fertilizer3, c_target)

print("------------------------------------------------------------------------------------------")
print(f"{minimized_value = }")
print("------------------------------------------------------------------------------------------")
print(f"{[a_opt, b_opt, c_opt] = }")
print("------------------------------------------------------------------------------------------")
print(f"Nutrient excess:")
print(f"{nut.round(3) = }")

NameError: name 'maxes' is not defined

## Iterative process

The next step is to calculate a schedule how to fertilize the plants.
As input we give the time intervall that we want to attent to the plants.
As a ouput we want the amount of each fertilizer we have to add for each time.

Comments:
- additional statistic would be nice
- some plots would be also nice

In [31]:
def calculate_fertilization_schedule(data_path, time_intervall_days, c_fertilizer1, c_fertilizer2, c_fertilizer3):
    
    # read and normalize data
    data, maxes = process_data(data_path=data_path)
    
    # normalize fartilizer
    norm_c_fertilizer1 = c_fertilizer1/maxes
    norm_c_fertilizer2 = c_fertilizer2/maxes
    norm_c_fertilizer3 = c_fertilizer3/maxes

    # interpolate the data
    x_interp, Y_interp = interpolate_data(data, num_points=4*7, kind='linear')
    # calculate the uptake of each nutrient of the plant
    # x_interp, plant_uptake_rate = calculate_nutrient_uptake_rate(x_interp, Y_interp, num_interp_points=500)
    # calculate 
    time_intervall_week = time_intervall_days/7
    num_fertilization_events = int(x_interp[-1] // time_intervall_week)

    residual = 0
    results = dict()

    # build schedule
    for i in range(num_fertilization_events):
        
        start = i*time_intervall_week
        end = (i+1)*time_intervall_week

        c_target_without_residuals = absolute_plant_uptake_during_interval(x_interp=x_interp, Y_interp=Y_interp, start=start ,end=end)

        # calculate the target considering residual (excess) nutrients in the solution
        c_target = np.abs(c_target_without_residuals - residual)

        # Define the constraint
        constraint = {
            'type': 'ineq',  # 'ineq' means the constraint function must return >= 0
            'fun': constraint_function,
            'args': (norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target)
        }

        # optimize 
        a_opt, b_opt, c_opt, minimized_value = optimize(norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target, constraint)
        
        params_opt = (a_opt, b_opt, c_opt)
        
        # calculate the excess nutrient after time period
        residual = nutrients(params_opt, norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target)

        # store the results
        results[f"{int(start*7)}"] = {'f1' : a_opt,
                                     'f2' : b_opt,
                                     'f3' : c_opt,
                                     'excess_1': residual[0],
                                     'excess_2': residual[1],
                                     'excess_3': residual[2],
                                     'excess_4': residual[3],
                                     'excess_5': residual[4],
                                     'excess_6': residual[5],
                                     'excess_7': residual[6],
                                     'excess_8': residual[7],
                                     'excess_9': residual[8],
                                     'excess_10': residual[9],
                                     'excess_11': residual[10],
                                    #  'excess' : (residual*maxes).round(3),
                                    #  'excess_normalized' : (residual).round(3),
                                     'objective_function': minimized_value}
    df = pd.DataFrame(results).T
    return df

c_fertilizer1 = np.abs(np.random.sample(11))*0.01
c_fertilizer2 = np.abs(np.random.sample(11))*0.01
c_fertilizer3 = np.abs(np.random.sample(11))*0.01
   

results = calculate_fertilization_schedule(data_path=DATA_PATH_AVG, time_intervall_days=1, c_fertilizer1=c_fertilizer1, c_fertilizer2=c_fertilizer2, c_fertilizer3=c_fertilizer3)

results.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/input/nutrients_avg.xlsx'

## Analyse results

In [129]:
def plot_excess_stacked(df_transposed):
    excess_labels = [f'Excess {i}' for i in range(len(df_transposed['excess'].iloc[0]))]
    days = df_transposed.index
    day_array = [int(day.split('_')[0])  for day in days]
    x = np.arange(len(days))  # the label locations for days
    width = 0.35  # The width of the bars

    # Plot stacked bar chart
    plt.figure(figsize=(10, 6))

    # Initialize the bottom of the stack
    bottom = np.zeros(len(days))

    # Plot each excess group on top of the previous one (stacked)
    for i, label in enumerate(excess_labels):
        excess_values = [df_transposed.loc[day, 'excess'][i] for day in df_transposed.index]
        plt.bar(x, excess_values, width=width, label=label, bottom=bottom)
        
        # Update the bottom to include the current excess values
        bottom += excess_values

    # Adding gridlines
    plt.grid(axis='y', which='both', linestyle='--', linewidth=0.7, color='grey')

    # Labeling
    plt.xlabel('Days')
    plt.ylabel('Excess Value')
    plt.title('Stacked Excess Values for Different Days')

    # Adjust x-ticks to display numeric labels (1, 2, 3, ...) instead of day_0, day_7, etc.
    # plt.xticks(x, labels=[i for i in range(len(days))])
    x_labels = [day+day_array[1] for day in day_array]
    plt.xticks(x, labels=x_labels)

    # Add legend and show the plot
    plt.legend(title='Excess Groups')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_excess_stacked(results)

In [ ]:
results['objective_function']